In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import xarray as xr
import regionmask

Data from POWER Data Access Viewer - NASA (1997-2020)

In [3]:
da1 = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\POWER_Regional_monthly_1997_2020.nc"
da2 = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\POWER_Regional_monthly_1997_2020_south.nc"

file_paths_list =[da1,da2]
monthly_forecast=xr.Dataset()

for file in file_paths_list:
        monthly_forecast = xr.merge([monthly_forecast,xr.open_mfdataset(file)], compat='no_conflicts')

Data from UEA CRU TS4.07 (1991-2022)

In [19]:
file = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\cru_ts4.07.1901.2022.tmx.dat.nc"
file2 = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\cru_ts4.07.1901.2022.pre.dat.nc"

file_paths_list =[file,file2]
monthly_forecast=xr.Dataset()

for file in file_paths_list:
        monthly_forecast = xr.merge([monthly_forecast,xr.open_mfdataset(file)], compat='override')

Shapefile with administrative boundaries of Somalia

In [6]:
path = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\som_adm_ocha_itos_20230308_shp\som_admbnda_adm1_ocha_20230308.shp"
states_gdf = gpd.read_file(path) 

Limit the lat-lon and time 

In [21]:
def get_aoi(shp, world=True):
    lon_lat = {}
    # Get lat min, max
    aoi_lat = [float(shp.total_bounds[1]), float(shp.total_bounds[3])]
    aoi_lon = [float(shp.total_bounds[0]), float(shp.total_bounds[2])]

    lon_lat["lon"] = aoi_lon
    lon_lat["lat"] = aoi_lat
    return lon_lat

bounds = get_aoi(regions_nev)

In [1]:
start_date = '1901-01-16'
end_date = '2022-12-16'

region = monthly_forecast[["pre",'tmx']].sel(
    time=slice(start_date, end_date),
    lon=slice(bounds["lon"][0], bounds["lon"][1]),
    lat=slice(bounds["lat"][0], bounds["lat"][1]))

NameError: name 'monthly_forecast' is not defined

In [52]:
region_mask = regionmask.mask_3D_geopandas(states_gdf,
                                         monthly_forecast.lon,
                                         monthly_forecast.lat)

somalia = region.where(region_mask).groupby("time").mean(["lat", "lon"]).to_dataframe().reset_index()

In [53]:
replacement_dict = {0  :  'Awdal',
1    :         'Bakool',
2      :       'Banadir',
3      :         'Bari',
4       :         'Bay',
5        :  'Galgaduud',
6          :      'Gedo',
7          :   'Hiraan',
8   :       'Lower Juba',
9   :   'Lower Shabelle',
10  :      'Middle Juba',
11   : 'Middle Shabelle',
12    :          'Mudug',
13    :        'Nugaal',
14      :       'Sanaag',
15       :        'Sool',
16        :   'Togdheer',
17   : 'Woqooyi Galbeed'}

In [55]:
somalia['region'] = somalia['region'].replace(replacement_dict)

In [61]:
somalia.to_csv('temp_pre_somalia.csv', index=False)